In [1]:
import os 
import tensorflow 
from PIL import Image, ImageDraw
import math
import pandas as pd 
from tqdm import tqdm 


In [2]:
folder_sources = []
Orig_Dataset_path = "Segmented Herbal Leaf images"
New_Dataset_path = "Transformed"

for file in os.listdir(Orig_Dataset_path):
    folder_sources.append(file)
        
print(folder_sources)

['Mentha (Mint)']


In [4]:
for folder in folder_sources:
    f_ = os.path.join(New_Dataset_path, folder)
    if not os.path.exists(f_):
        os.mkdir(f_)
        print("Created Path:", f_)

In [3]:
# Degrees is in radians
def calculate_new_coords(degrees, x, y, iw, ih):
    res_x = iw / 2 + (x - iw / 2) * math.cos(degrees) + (y - ih / 2) * math.sin(degrees)
    res_y = ih / 2 - (x - iw / 2) * math.sin(degrees) + (y - ih / 2) * math.cos(degrees)
    return (res_x, res_y)

In [5]:
BASE_IMAGE_SIZE = ((100, 100))
BASE_CANVAS_SIZE = ((200, 200))
X_VARIATIONS = 5
Y_VARIATIONS = 5
ROTATION_VARIATION = 36
ROTATION_DEGREES = 360 / ROTATION_VARIATION
DRAW_BOUNDING_BOX = False
BORDER_WIDTH = 5 
BORDER_COLOR = "Blue"

In [6]:
for folder in tqdm(folder_sources):
    current_folder_dict = {
        "image_name" : [],
        "x0" : [],
        "y0" : [],
        "x1" : [],
        "y1" : [],
        "x2" : [],
        "y2" : [],
        "x3" : [],
        "y3" : [],
        "class" : []
    }

    for image in os.listdir(os.path.join(Orig_Dataset_path, folder)):
        print(image)

        current_image = Image.open(os.path.join(Orig_Dataset_path, folder, image))
        current_image.thumbnail(BASE_IMAGE_SIZE)
        
        x0, y0 = (0, 0)
        x1, y1 = current_image.size 
        iw, ih = current_image.size 
        
        # Rotate Image
        for i in range(ROTATION_VARIATION + 1):
            current_degrees = int(i * ROTATION_DEGREES)

            rotated_image = current_image.copy()
            rotated_image = rotated_image.rotate(current_degrees, expand=True)

            r_i_width, r_i_height = rotated_image.size

            original_bbox_coords = [(x0, y0), (x1, y0), (x1, y1), (x0, y1)]

            _degrees = math.radians(current_degrees)

            new_coords = []
            _x, _y = 0, 1
            
            for x, y in original_bbox_coords:
                __x, __y = calculate_new_coords(_degrees, x, y, iw, ih)
                _x = min(__x, _x)
                _y = max(__y, _y)
                new_coords.append((__x, __y))

            excess_x = 0 - _x 
            excess_y = r_i_height - _y
            
            final_bbox_coords = [(x + excess_x, y + excess_y) for x, y in new_coords]
                
            # Transposition
            c_w , c_h = BASE_CANVAS_SIZE

            x_steps = abs(r_i_width - c_w) / X_VARIATIONS
            y_steps = abs(r_i_height - c_h) / Y_VARIATIONS
            
            for y in range(Y_VARIATIONS + 1):
                for x in range(X_VARIATIONS + 1):
                    canvas = Image.new("RGB", BASE_CANVAS_SIZE)
                    x_movement, y_movement = int(x_steps) * x, int(y_steps) * y
                    canvas.paste(rotated_image, (x_movement, y_movement))
                    

                    transposed_bbox_coords = [(x + x_movement, y + y_movement) for x, y in final_bbox_coords]

                    if DRAW_BOUNDING_BOX:
                        _d = ImageDraw.Draw(canvas)
                        _d.polygon(transposed_bbox_coords, width=BORDER_WIDTH, outline=BORDER_COLOR)

                    #display(canvas)

                    c0, c1, c2, c3 = transposed_bbox_coords
                    _x0, _y0 = c0 
                    _x1, _y1 = c1
                    _x2, _y2 = c2
                    _x3, _y3 = c3

                    current_folder_dict["image_name"].append(image)
                    current_folder_dict["x0"].append(_x0 / c_w) 
                    current_folder_dict["y0"].append(_y0 / c_h)
                    current_folder_dict["x1"].append(_x1 / c_w) 
                    current_folder_dict["y1"].append(_y1 / c_h)
                    current_folder_dict["x2"].append(_x2 / c_w) 
                    current_folder_dict["y2"].append(_y2 / c_h)
                    current_folder_dict["x3"].append(_x3 / c_w)
                    current_folder_dict["y3"].append(_y3 / c_h)
                    current_folder_dict["class"].append(folder)
                    
                    image_name = f"{New_Dataset_path}/{folder}/{image}_{current_degrees}_{x}_{y}.jpg"
                    canvas.save(image_name)

                    # Only 1
                    # break              
                # Only 1
                # break         
            # Only 1 Rotation
            # break
        # Test Only 1 Picture
        # break 
    
    df_current_folder = pd.DataFrame(current_folder_dict)
    print(df_current_folder.head())
    df_current_folder.to_csv(f"{New_Dataset_path}/{folder}/annotations.csv")

    # Test Only 1 Folder
    # break 


  0%|          | 0/1 [00:00<?, ?it/s]

1.jpg
10.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
2.jpg
20.jpg
21.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
3.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
4.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.jpg
5.jpg
50.jpg
6.jpg
8.jpg
9.jpg
  image_name   x0   y0   x1   y1   x2     y2   x3     y3          class
0      1.jpg  0.0  0.0  0.5  0.0  0.5  0.375  0.0  0.375  Mentha (Mint)
1      1.jpg  0.1  0.0  0.6  0.0  0.6  0.375  0.1  0.375  Mentha (Mint)
2      1.jpg  0.2  0.0  0.7  0.0  0.7  0.375  0.2  0.375  Mentha (Mint)
3      1.jpg  0.3  0.0  0.8  0.0  0.8  0.375  0.3  0.375  Mentha (Mint)
4      1.jpg  0.4  0.0  0.9  0.0  0.9  0.375  0.4  0.375  Mentha (Mint)


100%|██████████| 1/1 [01:33<00:00, 93.83s/it]


In [7]:
for folder in os.listdir(New_Dataset_path):
    for file in os.listdir(os.path.join(New_Dataset_path, folder)):
        if ".csv" in file:
            print(file)

annotations.csv
annotations.csv
annotations.csv
annotations.csv
annotations.csv
annotations.csv
annotations.csv
annotations.csv
annotations.csv
annotations.csv
